In [14]:
%load_ext autoreload
%autoreload 2
import collections
import os
from keras.preprocessing import text
import argparse,sys
import numpy as np
import pickle
import pandas as pd
from dotmap import DotMap

import numpy as np
import pandas as pd
import io
import numpy as np
import subprocess
import scipy as sp
from sklearn.metrics import  mean_squared_error, r2_score

import keras
from keras.preprocessing import sequence
from keras.layers import merge, Embedding, Bidirectional, TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.core import *
from keras.models import *
from keras.layers.pooling import MaxPooling1D
from keras.layers.recurrent import GRU, LSTM
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import *
import dotmap

import xgboost as xgb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from feature_util import *
def my_feature(df_model,learn_options):
    '''
    Args:
        df_model:DataFrame数据，必须包含21mer信息
        learn_options:训练模型、生成特征所需要的参数
    returns: inputs,返回的是numpy array特征
    '''
    feature_sets = {}
    feature_sets = featurize_data(df_model, learn_options)
    inputs, dim, dimsum, feature_names = concatenate_feature_sets(feature_sets)
    return inputs,dim,dimsum,feature_names,feature_sets

In [16]:
def change_to_1(y):
    y[y > 1] = 1
    y[y < 0] = 0
    return y

In [10]:
with open('../../Thesis/data/wt_feat_array_data.pkl', 'rb') as handle:
    X_selected,y,features = pickle.load(handle)

In [15]:
with open('../../Thesis/data/esp_seq_data_array.pkl', 'rb') as handle:
    test = pickle.load(handle)

In [48]:
#load data
with open('data/esp_seq_data_array.pkl', 'rb') as handle:
    esp_data = pickle.load(handle)
with open('data/hf_seq_data_array.pkl', 'rb') as handle:
    hf_data = pickle.load(handle)
with open('data/wt_seq_data_array.pkl', 'rb') as handle:
    wt_data = pickle.load(handle)

In [49]:
X, X_biofeat, y = esp_data['X_1'],esp_data['X_biofeat'],esp_data['y']

In [19]:
from feature_util import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,ShuffleSplit,StratifiedKFold,GroupKFold
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import cross_val_score


from sklearn import linear_model
import GPy
import GPyOpt
import matplotlib.pyplot as plt

In [20]:
import numpy as np
from keras.callbacks import Callback
class GetBest(Callback):
    """Get the best model at the end of training.
	# Arguments
        monitor: quantity to monitor.
        verbose: verbosity mode, 0 or 1.
        mode: one of {auto, min, max}.
            The decision
            to overwrite the current stored weights is made
            based on either the maximization or the
            minimization of the monitored quantity. For `val_acc`,
            this should be `max`, for `val_loss` this should
            be `min`, etc. In `auto` mode, the direction is
            automatically inferred from the name of the monitored quantity.
        period: Interval (number of epochs) between checkpoints.
	# Example
		callbacks = [GetBest(monitor='val_acc', verbose=1, mode='max')]
		mode.fit(X, y, validation_data=(X_eval, Y_eval),
                 callbacks=callbacks)
    """

    def __init__(self,filepath=None, monitor='val_loss', save_best=False,verbose=0,
                 mode='auto', period=1):
        super(GetBest, self).__init__()
        self.monitor = monitor
        self.verbose = verbose
        self.period = period
        self.save_best = save_best
        self.filepath = filepath
        self.best_epochs = 0
        self.epochs_since_last_save = 0

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('GetBest mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor or self.monitor.startswith('fmeasure'):
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf
                
    def on_train_begin(self, logs=None):
        self.best_weights = self.model.get_weights()

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch=epoch + 1, **logs)
            current = logs.get(self.monitor)
            if current is None:
                warnings.warn('Can pick best model only with %s available, '
                              'skipping.' % (self.monitor), RuntimeWarning)
            else:
                if self.monitor_op(current, self.best):
                    if self.verbose > 0:
                        print('\nEpoch %05d: %s improved from %0.5f to %0.5f,'
                              ' storing weights.'
                              % (epoch + 1, self.monitor, self.best,
                                 current))
                    self.best = current
                    self.best_epochs = epoch + 1
                    self.best_weights = self.model.get_weights()
                    #self.model.save(filepath, overwrite=True)
                else:
                    if self.verbose > 0:
                        print('\nEpoch %05d: %s did not improve.' %
                              (epoch + 1, self.monitor)) 
                    
    def on_train_end(self, logs=None):
        if self.verbose > 0:
            print('Using epoch %05d with %s: %0.5f.' % (self.best_epochs, self.monitor,
                                                       self.best))
        self.model.set_weights(self.best_weights)
        #self.model.save(filepath, overwrite=True)

In [21]:
def load_data_kf(X,y,split=0.15):
    train_test_data = []
    kf = ShuffleSplit(n_splits=10, test_size=0.15, random_state=33)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        train_test_data.append((X_train,X_test,y_train,y_test))
    return train_test_data

### XGBoost

In [29]:
import pickle
with open('data/esp_feat_data.pkl', 'rb') as f:
    data_list = pickle.load(f)

In [30]:
X,y = data_list[0], np.array(data_list[1])
feat_indext = np.loadtxt('data/esp_feat_index.txt').astype('int')
X_selected = X[:,feat_indext]

In [ ]:
data_list = load_data_kf(X,np.array(y))
training_r2_scores = []
testing_r2_scores = []
spearmanrs = []
loss = []
models = []
for data in data_list:
    X_train,X_test, y_train,y_test = data[0],data[1],data[2],data[3]
    data_train = xgb.DMatrix( X_train, y_train )
    data_test = xgb.DMatrix( X_test, y_test )
    param = {'learning_rate': 0.12, 'subsample': 0.7222222222222222, 'colsample_bytree': 0.6666666666666666, 
             'max_depth': 5, 'lambda': 700, 'objective': 'reg:squarederror', 
             'nthread': 18, 'booster': 'gbtree', 'seed': 1771}
    watch_list = [(data_test, 'eval'), (data_train, 'train')]
    bst = xgb.train( param, data_train, num_boost_round=700 ,evals=watch_list,early_stopping_rounds=3 )

    y_train_pred = bst.predict(data_train)
    y_test_pred = bst.predict(data_test)
    raining_score = r2_score( y_train, y_train_pred )
    testing_score = r2_score( y_test, y_test_pred )
    spearmanr = sp.stats.spearmanr(y_test, y_test_pred)[0]   
    mse = mean_squared_error(y_test, y_test_pred)

    training_r2_scores.append(raining_score)
    testing_r2_scores.append(testing_score)
    spearmanrs.append(spearmanr)
    loss.append(mse)
    models.append(bst)

In [26]:
r = {}
r['training_r2_scores'] = np.mean(training_r2_scores), np.std(training_r2_scores)
r['testing_r2_scores'] = np.mean(testing_r2_scores),np.std(testing_r2_scores)
r['spearmanr'] = np.mean(spearmanrs), np.std(spearmanrs)
r['loss'] = np.mean(loss),np.std(loss)
r

{'training_r2_scores': (0.7369090120914517, 0.0013484501623500602),
 'testing_r2_scores': (0.6741602042722457, 0.008407645462466747),
 'spearmanr': (0.8309723102248828, 0.004579773609764182),
 'loss': (0.011503698611460831, 0.00033907608093433015)}

In [28]:
spearmanrs

[0.8320818569005929,
 0.8286785617873859,
 0.840044090263017,
 0.8260184428428017,
 0.8354593897202929,
 0.8290010171733062,
 0.8358978872735134,
 0.8279861515661779,
 0.8248368973183632,
 0.8297188074033771]

In [10]:
r = {}
r['training_r2_scores'] = np.mean(training_r2_scores), np.std(training_r2_scores)
r['testing_r2_scores'] = np.mean(testing_r2_scores),np.std(testing_r2_scores)
r['spearmanr'] = np.mean(spearmanrs), np.std(spearmanrs)
r['loss'] = np.mean(loss),np.std(loss)
r

{'training_r2_scores': (0.7369090120914517, 0.0013484501623500602),
 'testing_r2_scores': (0.6741602042722457, 0.008407645462466747),
 'spearmanr': (0.8309723102248828, 0.004579773609764182),
 'loss': (0.011503698611460831, 0.00033907608093433015)}

In [12]:
spearmanrs

[0.8320818569005929,
 0.8286785617873859,
 0.840044090263017,
 0.8260184428428017,
 0.8354593897202929,
 0.8290010171733062,
 0.8358978872735134,
 0.8279861515661779,
 0.8248368973183632,
 0.8297188074033771]

In [44]:
r = {}
r['training_r2_scores'] = np.mean(training_r2_scores), np.std(training_r2_scores)
r['testing_r2_scores'] = np.mean(testing_r2_scores),np.std(testing_r2_scores)
r['spearmanr'] = np.mean(spearmanrs), np.std(spearmanrs)
r['loss'] = np.mean(loss),np.std(loss)
r

{'training_r2_scores': (0.7371195937572591, 0.0012023099783800063),
 'testing_r2_scores': (0.6749394827318866, 0.008179213083283057),
 'spearmanr': (0.8315335878103474, 0.004797317859471297),
 'loss': (0.011475680780733247, 0.0003139780377941402)}

In [45]:
spearmanrs

[0.8307690773859965,
 0.8286083782226864,
 0.840217307114493,
 0.8252908178026062,
 0.8370227754990576,
 0.8281813156049919,
 0.838068583140455,
 0.8303111506617731,
 0.8274771204158126,
 0.8293893522556008]

### Linear Regression

In [35]:
data_list = load_data_kf(X_selected,y)
training_r2_scores = []
testing_r2_scores = []
spearmanrs = []
loss = []
for data in data_list:
    X_train,X_test, y_train,y_test = data[0],data[1],data[2],data[3]
    model = linear_model.Ridge(alpha=104.32718311764306, fit_intercept=True, 
                             normalize=False, copy_X=True, max_iter=None, tol=0.001, solver='auto')
    #model = linear_model.LinearRegression(n_jobs=20)
    model.fit(X_train, y_train)

    y_train_pred = change_to_1(model.predict(X_train))
    y_test_pred = change_to_1(model.predict(X_test))
    mse = mean_squared_error(y_test, y_test_pred)

    training_r2_scores.append(r2_score( y_train, y_train_pred ))
    testing_r2_scores.append(r2_score( y_test, y_test_pred ))
    spearmanrs.append(sp.stats.spearmanr(y_test,y_test_pred)[0])
    loss.append(mse)

In [36]:
r = {}
r['training_r2_scores'] = np.mean(training_r2_scores), np.std(training_r2_scores)
r['testing_r2_scores'] = np.mean(testing_r2_scores),np.std(testing_r2_scores)
r['spearmanr'] = np.mean(spearmanrs), np.std(spearmanrs)
r['loss'] = np.mean(loss),np.std(loss)
r

{'training_r2_scores': (0.5826840011493762, 0.0017798479183876385),
 'testing_r2_scores': (0.5795174214005189, 0.010104929383200612),
 'spearmanr': (0.7698081619705687, 0.006797405517946372),
 'loss': (0.014844643106943823, 0.0003997905148501007)}

In [38]:
for i in spearmanrs:
    print(i)

0.7703405622032071
0.7643935412214841
0.7848826572358722
0.7625796922429551
0.7775308642228411
0.7660045188102381
0.7731737706147075
0.7694589738140001
0.761825413613909
0.7678916257264724


In [32]:
r = {}
r['training_r2_scores'] = np.mean(training_r2_scores), np.std(training_r2_scores)
r['testing_r2_scores'] = np.mean(testing_r2_scores),np.std(testing_r2_scores)
r['spearmanr'] = np.mean(spearmanrs), np.std(spearmanrs)
r['loss'] = np.mean(loss),np.std(loss)
r

{'training_r2_scores': (0.5835083424678227, 0.001777749934034861),
 'testing_r2_scores': (0.5801290776107423, 0.010119372894777011),
 'spearmanr': (0.7701467356681887, 0.006758787527908767),
 'loss': (0.014823066127737922, 0.00040068168703661196)}

In [33]:
spearmanrs

[0.7707725234945665,
 0.7647773642550586,
 0.7851534970130917,
 0.7627835344362617,
 0.7777387271831084,
 0.7662933996396077,
 0.7733326605667139,
 0.7700689325466341,
 0.7622709929529221,
 0.7682757245939218]

### MLP Model

In [39]:
def mlp_model(fc_drop=0.4, batch_size=60, epochs=35, fc_num_hidden_layers=5, fc_num_units=100, fc_activation='3', optimizer='5'):
    fc_activation = fc_activation_dict[str(fc_activation)]
    optimizer = optimizer_dict[str(optimizer)]
    
    feat_input = Input(name = 'feat_input', shape = (X_train.shape[1],))
    x = feat_input
    for l in range(fc_num_hidden_layers):
        x = Dense(fc_num_units, activation=fc_activation)(x)
        x = Dropout(fc_drop)(x)


    output = Dense(1, activation='linear',name='output')(x)
    model = Model(inputs=[feat_input], outputs=[output])
    model.compile(loss='mse',optimizer=optimizer(lr=0.001))

    np.random.seed(1337)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    get_best_model = GetBest('models/esp_mlp.hd5',monitor='val_loss', verbose=1, mode='min')
    model.fit([X_train], 
                 y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=2,
                 validation_split=0.1,
                 shuffle=False,
                 callbacks=[get_best_model, early_stopping])
        
    return model

In [40]:
fc_activation_dict = {'1':'relu','2':'tanh', '3':'sigmoid', '4':'hard_sigmoid', '0':'elu'}
initializer_dict = {'1':'lecun_uniform','2':'normal', '3':'he_normal', '0':'he_uniform'}
optimizer_dict = {'1':SGD,'2':RMSprop, '3':Adagrad, '4':Adadelta,'5':Adam,'6':Adamax,'0':Nadam}

In [41]:
param = {'fc_drop':0.2,
'batch_size':35,
'epochs':40,
'fc_num_hidden_layers':4,
'fc_num_units':400,
'fc_activation':'3',
'optimizer':'6'}

In [ ]:
data_list = load_data_kf(X_selected,y)
training_r2_scores = []
testing_r2_scores = []
spearmanrs = []
loss = []
for data in data_list:
    X_train,X_test, y_train,y_test = data[0],data[1],data[2],data[3]
    model = mlp_model(**param)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_test_pred)

    training_r2_scores.append(r2_score( y_train, y_train_pred ))
    testing_r2_scores.append(r2_score( y_test, y_test_pred ))
    spearmanrs.append(sp.stats.spearmanr(y_test,y_test_pred)[0])
    loss.append(mse)

In [45]:
r = {}
r['training_r2_scores'] = np.mean(training_r2_scores), np.std(training_r2_scores)
r['testing_r2_scores'] = np.mean(testing_r2_scores),np.std(testing_r2_scores)
r['spearmanr'] = np.mean(spearmanrs), np.std(spearmanrs)
r['loss'] = np.mean(loss),np.std(loss)
r

{'training_r2_scores': (0.7632138624587748, 0.01659869723608099),
 'testing_r2_scores': (0.7018382546946994, 0.00774254308643637),
 'spearmanr': (0.8457228621789528, 0.0029497993972957327),
 'loss': (0.01052690595414275, 0.0003238017010849953)}

In [47]:
for i in loss:
    print(i)

0.010127593280284373
0.010238676220832146
0.009957035049880806
0.010920643572781979
0.010474505169919058
0.010830294123300303
0.010650946325839293
0.010799664865294866
0.010878046446646887
0.010391654486647793


In [44]:
for i in spearmanrs:
    print(i)

[0.8487986374203252,
 0.847841295231162,
 0.8490629771573187,
 0.8435228616038357,
 0.8475177601906175,
 0.8411522181604375,
 0.8451220319063756,
 0.8458054089697685,
 0.8408907471613889,
 0.845528688256899]

### CNN

In [50]:
def load_data_kf(X,X_biofeat,y,split=0.15):
    train_test_data = []
    kf = ShuffleSplit(n_splits=10, test_size=0.15, random_state=33)
    for train_index, test_index in kf.split(X):
        X_train, X_train_biofeat, X_test, X_test_biofeat = X[train_index],X_biofeat[train_index], X[test_index],X_biofeat[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        train_test_data.append((X_train,X_train_biofeat,X_test,X_test_biofeat,y_train,y_test))
    return train_test_data

In [55]:
def cnn_model(batch_size=90, epochs=50, initializer='0',em_dim=44,em_drop=0.2,cnn_config=None,
                fc_num_hidden_layers=3,fc_num_units=320, fc_drop=0.4,fc_activation='elu',optimizer=Adam,
                learning_rate=0.001, validation_split=0.1,shuffle=False):
    fc_activation = fc_activation_dict[str(fc_activation)]
    cnn_config = cnn_config_dict[str(cnn_config)]
    optimizer = optimizer_dict[str(optimizer)]
    sequence_input = Input(name = 'seq_input', shape = (22,))

    embedding_layer = Embedding(7,em_dim,input_length=22)
    embedded = embedding_layer(sequence_input)
    embedded = SpatialDropout1D(em_drop)(embedded)
    x = embedded
    
    
    
    #CNN
    convs = []
    kernel_sizes = [int(f) for f in cnn_config[0].split(':')]
    feature_maps = [int(f) for f in cnn_config[1].split(':')]
    #构建多层卷积神经网络
    for i in range(len(feature_maps)):
        conv_layer = Conv1D(filters=feature_maps[i],
                            kernel_size=kernel_sizes[i],
                            padding='same',
                            activation='relu',
                            strides=1)
        conv_out = conv_layer(x)
        pool_out = MaxPooling1D(pool_size=conv_layer.output_shape[1])(conv_out)
        flatten_out = Flatten()(pool_out)

        convs.append(flatten_out)

    x = keras.layers.concatenate(convs) if len(convs) > 1 else convs[0]

#     #RNN
#     lstm = LSTM(rnn_units, dropout=rnn_drop, 
#                 kernel_regularizer='l2',recurrent_regularizer='l2',
#                 recurrent_dropout=rnn_rec_drop, return_sequences=True)
#     x = Bidirectional(lstm)(x)
#     x = Flatten()(x)

#     #生物学特征
#     biological_input = Input(name = 'bio_input', shape = (X_train_biofeat.shape[1],))
#     x = keras.layers.concatenate([x, biological_input])


    for l in range(fc_num_hidden_layers):
        x = Dense(fc_num_units, activation=fc_activation)(x)
        x = Dropout(fc_drop)(x)
    #finish model
    mix_output = Dense(1, activation='linear',name='mix_output')(x)

    #model = Model(inputs=[sequence_input, biological_input], outputs=[mix_output])
    model = Model(inputs=[sequence_input], outputs=[mix_output])
    model.compile(loss='mse', optimizer=optimizer(lr=0.001))
    
    np.random.seed(1337)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    get_best_model = GetBest('Model/esp_cnn.hd5',monitor='val_loss', verbose=1, mode='min')
    #model.fit([X_train,X_train_biofeat], 
    model.fit([X_train], 
                 y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=2,
                 validation_split=0.1,
                 shuffle=False,
                 callbacks=[get_best_model, early_stopping])    
    return model

In [56]:
config0 = ['1:2:3:4:5:6:7:8:9:10:11:12:13:14:15','20:20:20:20:20:20:20:20:20:20:20:20:20:20']
config1 = ['1:2:3:4:5:6:7:8:9:10:11:12:13:14:15','20:20:20:20:20:20:20:40:40:40:40:40:40:40']
config2 = ['1:2:3:4:5:6:7:8:9:10:11:12:13:14:15','20:20:20:20:40:40:40:40:80:80:80:80:80:80']
config3 = ['1:2:3:4:5:6:7:8:9:10:11:12:13:14:15','40:40:40:40:40:40:40:40:80:80:80:80:80:80']


cnn_config_dict = {'1':config1,'2':config2, '3':config3,'0':config0}

In [57]:
param = {'em_drop': 0.2, 'fc_drop': 0.2, 'batch_size': 70, 'epochs': 30, 'em_dim': 36, 'cnn_config': 2, 
         'fc_num_hidden_layers': 3, 'fc_num_units': 400, 'fc_activation': 1, 'optimizer': 6}

#model_rnn = lstm_model(**param)

In [58]:
X, X_biofeat, y = esp_data['X_1'],esp_data['X_biofeat'],esp_data['y']

In [ ]:

data_list = load_data_kf(X,X_biofeat,y)
training_r2_scores = []
testing_r2_scores = []
spearmanrs = []
loss = []
for data in data_list:
    X_train,X_train_biofeat,X_test,X_test_biofeat,y_train,y_test = data[0],data[1],data[2],data[3],data[4],data[5]
    model = cnn_model(**param)
    #y_train_pred = model.predict([X_train,X_train_biofeat])
    #y_test_pred = model.predict([X_test,X_test_biofeat])
    y_train_pred = model.predict([X_train])
    y_test_pred = model.predict([X_test])
    training_score = r2_score( y_train, y_train_pred )
    testing_score = r2_score( y_test, y_test_pred )
    mse = mean_squared_error( y_test, y_test_pred)
    spearmanr = sp.stats.spearmanr(y_test, y_test_pred)[0] 

    training_r2_scores.append(training_score)
    testing_r2_scores.append(testing_score)
    spearmanrs.append(spearmanr)
    loss.append(mse)

In [60]:
r = {}
r['training_r2_scores'] = np.mean(training_r2_scores), np.std(training_r2_scores)
r['testing_r2_scores'] = np.mean(testing_r2_scores),np.std(testing_r2_scores)
r['spearmanr'] = np.mean(spearmanrs), np.std(spearmanrs)
r['loss'] = np.mean(loss),np.std(loss)
r

{'training_r2_scores': (0.759248989106773, 0.032414447233595274),
 'testing_r2_scores': (0.6804897327919355, 0.016078851768164662),
 'spearmanr': (0.8313350706208231, 0.008840303247433087),
 'loss': (0.011279017406023512, 0.0005652284412441146)}

In [62]:
for i in loss:
    print(i)

0.01115373919841762
0.010994457931076224
0.010521041421004455
0.011298116857012113
0.010966085181726634
0.01274333305749055
0.01092944711277399
0.011394402989552918
0.011638507550669802
0.011151042760510805


In [59]:
spearmanrs

[0.8310248784708546,
 0.8331721285102813,
 0.8407636497220001,
 0.8324440201960122,
 0.8377643762402487,
 0.8320308109795865,
 0.8383716995210792,
 0.8282071518244732,
 0.8332817282145238,
 0.8306395585490115]

### LSTM

In [63]:
def lstm_model(batch_size=90, epochs=50, initializer='0',em_dim=44,em_drop=0.2,
                rnn_units=60, rnn_drop=0.6, rnn_rec_drop=0.1, fc_num_hidden_layers=3,
                fc_num_units=320, fc_drop=0.4,fc_activation='elu',optimizer=Adam,learning_rate=0.001,
                validation_split=0.1,shuffle=False):
    #X_train,X_train_biofeat,y_train = data[0],data[1],data[2]
    fc_activation = fc_activation_dict[str(fc_activation)]
    initializer = initializer_dict[str(initializer)]
    optimizer = optimizer_dict[str(optimizer)]
    sequence_input = Input(name = 'seq_input', shape = (22,))

    embedding_layer = Embedding(7,em_dim,input_length=22)
    embedded = embedding_layer(sequence_input)
    embedded = SpatialDropout1D(em_drop)(embedded)
    x = embedded

    #RNN
    lstm = LSTM(rnn_units, dropout=rnn_drop, 
                kernel_regularizer='l2',recurrent_regularizer='l2',
                recurrent_dropout=rnn_rec_drop, return_sequences=True)
    x = Bidirectional(lstm)(x)
    x = Flatten()(x)

#     #生物学特征
#     biological_input = Input(name = 'bio_input', shape = (X_train_biofeat.shape[1],))
#     x = keras.layers.concatenate([x, biological_input])


    for l in range(fc_num_hidden_layers):
        x = Dense(fc_num_units, activation=fc_activation)(x)
        x = Dropout(fc_drop)(x)
    #finish model
    mix_output = Dense(1, activation='linear',name='mix_output')(x)
    #model = Model(inputs=[sequence_input, biological_input], outputs=[mix_output])
    model = Model(inputs=[sequence_input], outputs=[mix_output])
    model.compile(loss='mse', optimizer=optimizer(lr=0.001))
    
    np.random.seed(1337)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    get_best_model = GetBest('Model/esp_rnn.hd5',monitor='val_loss', verbose=1, mode='min')
    #model.fit([X_train,X_train_biofeat], 
    model.fit([X_train], 
                 y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=2,
                 validation_split=0.1,
                 shuffle=False,
                 callbacks=[get_best_model, early_stopping])    
    return model

In [64]:
param = {'em_drop': 0.2, 'rnn_drop': 0.2, 'rnn_rec_drop': 0.4, 'fc_drop': 0.4, 
         'batch_size': 70, 'epochs': 50, 'em_dim': 44, 'rnn_units': 70, 'fc_num_hidden_layers': 2, 
         'fc_num_units': 300, 'fc_activation': 3, 'optimizer': 6}

#r = lstm_model(**param)

In [ ]:

data_list = load_data_kf(X,X_biofeat,y)
training_r2_scores = []
testing_r2_scores = []
spearmanrs = []
loss = []
for data in data_list:
    X_train,X_train_biofeat,X_test,X_test_biofeat,y_train,y_test = data[0],data[1],data[2],data[3],data[4],data[5]
    model = lstm_model(**param)
    #y_train_pred = model.predict([X_train,X_train_biofeat])
    #y_test_pred = model.predict([X_test,X_test_biofeat])
    y_train_pred = model.predict([X_train])
    y_test_pred = model.predict([X_test])
    training_score = r2_score( y_train, y_train_pred )
    testing_score = r2_score( y_test, y_test_pred )
    mse = mean_squared_error( y_test, y_test_pred)
    spearmanr = sp.stats.spearmanr(y_test, y_test_pred)[0] 

    training_r2_scores.append(training_score)
    testing_r2_scores.append(testing_score)
    spearmanrs.append(spearmanr)
    loss.append(mse)

Train on 44841 samples, validate on 4983 samples
Epoch 1/50
 - 18s - loss: 0.2440 - val_loss: 0.0341

Epoch 00001: val_loss improved from inf to 0.03407, storing weights.
Epoch 2/50
 - 16s - loss: 0.0447 - val_loss: 0.0344

Epoch 00002: val_loss did not improve.
Epoch 3/50
 - 16s - loss: 0.0382 - val_loss: 0.0321

Epoch 00003: val_loss improved from 0.03407 to 0.03206, storing weights.
Epoch 4/50
 - 16s - loss: 0.0345 - val_loss: 0.0283

Epoch 00004: val_loss improved from 0.03206 to 0.02826, storing weights.
Epoch 5/50
 - 16s - loss: 0.0284 - val_loss: 0.0233

Epoch 00005: val_loss improved from 0.02826 to 0.02331, storing weights.
Epoch 6/50
 - 16s - loss: 0.0251 - val_loss: 0.0227

Epoch 00006: val_loss improved from 0.02331 to 0.02275, storing weights.
Epoch 7/50
 - 16s - loss: 0.0231 - val_loss: 0.0200

Epoch 00007: val_loss improved from 0.02275 to 0.02001, storing weights.
Epoch 8/50
 - 16s - loss: 0.0209 - val_loss: 0.0184

Epoch 00008: val_loss improved from 0.02001 to 0.01843

In [66]:
r = {}
r['training_r2_scores'] = np.mean(training_r2_scores), np.std(training_r2_scores)
r['testing_r2_scores'] = np.mean(testing_r2_scores),np.std(testing_r2_scores)
r['spearmanr'] = np.mean(spearmanrs), np.std(spearmanrs)
r['loss'] = np.mean(loss),np.std(loss)
r

{'training_r2_scores': (0.7829373950316538, 0.009870604260570705),
 'testing_r2_scores': (0.7097636412825155, 0.009920452627772193),
 'spearmanr': (0.8491014520769882, 0.004744518983541059),
 'loss': (0.010245622300480706, 0.0003493527744093642)}

In [68]:
for i in spearmanrs:
    print(i)

0.8492094220956927
0.8466984549954044
0.8548235819742712
0.8455017188061741
0.8559545327630784
0.8446366187809149
0.8560893272149397
0.8502925401813233
0.8427983791390835
0.8450099448190005


### RNN + Biofeat

In [69]:
def lstm_model(batch_size=90, epochs=50, initializer='0',em_dim=44,em_drop=0.2,
                rnn_units=60, rnn_drop=0.6, rnn_rec_drop=0.1, fc_num_hidden_layers=3,
                fc_num_units=320, fc_drop=0.4,fc_activation='elu',optimizer=Adam,learning_rate=0.001,
                validation_split=0.1,shuffle=False):
    fc_activation = fc_activation_dict[str(fc_activation)]
    initializer = initializer_dict[str(initializer)]
    optimizer = optimizer_dict[str(optimizer)]
    sequence_input = Input(name = 'seq_input', shape = (22,))

    embedding_layer = Embedding(7,em_dim,input_length=22)
    embedded = embedding_layer(sequence_input)
    embedded = SpatialDropout1D(em_drop)(embedded)
    x = embedded

    #RNN
    lstm = LSTM(rnn_units, dropout=rnn_drop, 
                kernel_regularizer='l2',recurrent_regularizer='l2',
                recurrent_dropout=rnn_rec_drop, return_sequences=True)
    x = Bidirectional(lstm)(x)
    x = Flatten()(x)

#     #生物学特征
    biological_input = Input(name = 'bio_input', shape = (X_train_biofeat.shape[1],))
    x = keras.layers.concatenate([x, biological_input])


    for l in range(fc_num_hidden_layers):
        x = Dense(fc_num_units, activation=fc_activation)(x)
        x = Dropout(fc_drop)(x)
    #finish model
    mix_output = Dense(1, activation='linear',name='mix_output')(x)

    model = Model(inputs=[sequence_input, biological_input], outputs=[mix_output])
    #model = Model(inputs=[sequence_input], outputs=[mix_output])
    model.compile(loss='mse', optimizer=optimizer(lr=0.001))
    
    np.random.seed(1337)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
    get_best_model = GetBest('Model/hf_rnn.hd5',monitor='val_loss', verbose=1, mode='min')
    model.fit([X_train,X_train_biofeat], 
    #model.fit([X_train], 
                 y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=2,
                 validation_split=0.1,
                 shuffle=False,
                 callbacks=[get_best_model, early_stopping])    
    return model

In [70]:
param = {'em_drop': 0.2, 'rnn_drop': 0.5, 
     'rnn_rec_drop': 0.5, 'fc_drop': 0.4, 
     'batch_size': 80, 'epochs': 46, 
     'em_dim': 48, 'rnn_units': 80, 
     'fc_num_hidden_layers': 2, 'fc_num_units': 300, 
     'fc_activation': 3, 'optimizer': 6}

In [ ]:
data_list = load_data_kf(X,X_biofeat,y)
training_r2_scores = []
testing_r2_scores = []
spearmanrs = []
loss = []
for data in data_list:
    X_train,X_train_biofeat,X_test,X_test_biofeat,y_train,y_test = data[0],data[1],data[2],data[3],data[4],data[5]
    model = lstm_model(**param)
    y_train_pred = model.predict([X_train,X_train_biofeat])
    y_test_pred = model.predict([X_test,X_test_biofeat])
#     y_train_pred = model.predict([X_train])
#     y_test_pred = model.predict([X_test])
    training_score = r2_score( y_train, y_train_pred )
    testing_score = r2_score( y_test, y_test_pred )
    mse = mean_squared_error( y_test, y_test_pred)
    spearmanr = sp.stats.spearmanr(y_test, y_test_pred)[0] 

    training_r2_scores.append(training_score)
    testing_r2_scores.append(testing_score)
    spearmanrs.append(spearmanr)
    loss.append(mse)

In [72]:
r = {}
r['training_r2_scores'] = np.mean(training_r2_scores), np.std(training_r2_scores)
r['testing_r2_scores'] = np.mean(testing_r2_scores),np.std(testing_r2_scores)
r['spearmanr'] = np.mean(spearmanrs), np.std(spearmanrs)
r['loss'] = np.mean(loss),np.std(loss)
r

{'training_r2_scores': (0.7966211082899555, 0.007287918270726685),
 'testing_r2_scores': (0.7305182005037902, 0.008202604337021425),
 'spearmanr': (0.8624301371241108, 0.004027002438459456),
 'loss': (0.009513115137466098, 0.0002937367060889057)}

In [75]:
for i in loss:
    print(i)

0.009402830687045416
0.009471316484137888
0.009014022720839334
0.009994259769559253
0.009237383604450075
0.00948051783547902
0.009347444273015558
0.009590112324740925
0.010016949831851762
0.009576313843541741


In [35]:
spearmanrs

[0.8623787246455864,
 0.8611237117010575,
 0.866830537667525,
 0.8567591763880864,
 0.8687672729678532,
 0.8614907966442155,
 0.86755667461855,
 0.8632395207054615,
 0.8574422015211627,
 0.8609042823635265]